In [ ]:
import pandas as pd
import numpy as np
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import nltk
import tkinter 
from tkinter.filedialog import askopenfilename
import joblib
from sklearn.feature_extraction.text import CountVectorizer 
import pickle
#on va utiliser sur le main  







def preprocess(text):
    #remove punctuations
    punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

    # Arabic stop words with nltk
    stop_words = stopwords.words()

    arabic_diacritics = re.compile("""
                                 ّ    | # Shadda
                                 َ    | # Fatha
                                 ً    | # Tanwin Fath
                                 ُ    | # Damma
                                 ٌ    | # Tanwin Damm
                                 ِ    | # Kasra
                                 ٍ    | # Tanwin Kasr
                                 ْ    | # Sukun
                                 ـ     # Tatwil/Kashida
                             """, re.VERBOSE)

    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    #remove repeating char
    text=re.sub(r'(.)\1+', r'\1', text) ##hadi zdtha l9itha
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text=re.sub("شش","ش",text) ##hadu li lt7t zdthum
    text=re.sub("يع يع يع","يع",text) #
    text=re.sub("ووو","و",text) #
    text=re.sub("ييي","ي",text) #
    text=re.sub("رر","ر",text) #
    text=re.sub("تت","ت",text) #

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
def countvectorizer():
    
    data.loc[data['Sentiment']=='Positive',"Sentiment"]=1  ##hna bdelt blasst positive ghatwli 1
    data.loc[data['Sentiment']=='Negative',"Sentiment"]=0##w hna 0
    ##splitting
    data_X=data["Feed"]
    data_Y=data["Sentiment"]
    ##convertion du text en nombre sans perdre d'informations
    CV=CountVectorizer()
    X_train, X_test, y_train, y_test = train_test_split(data_X,data_Y)
    x_traincv=CV.fit_transform(X_train)
    return x_traincv
    
def logistic_regression():
    from sklearn.linear_model import LogisticRegression
    
    # splitting the data into target and feature
    feature = data.Feed
    target = data.Sentiment
    # splitting into train and tests
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

    # make pipeline
    pipe = make_pipeline(TfidfVectorizer(),
                        LogisticRegression())
    # make param grid
    param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}

    # create and fit the model
    model = GridSearchCV(pipe, param_grid, cv=5)
    model.fit(X_train,Y_train)
    # make prediction and print accuracy
    prediction = model.predict(X_test)
    print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
    print(classification_report(Y_test, prediction))
    #pkl
    joblib.dump(model, 'logistic_regression_model.pkl')
    NB_spam_model = open('logistic_regression_model.pkl','rb')
    clf = joblib.load(NB_spam_model)
def classifier_bayes():
    from sklearn.naive_bayes import MultinomialNB
    data['label'] = data['Sentiment'].map({'Positive': 0, 'Negative': 1})
    feature=data["Feed"]
    target=data["label"]
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

    b_model= make_pipeline(TfidfVectorizer(),
                        MultinomialNB())
    b_model.fit(X_train,Y_train)
    prediction = b_model.predict(X_test)
    #pkl
    print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
    print(classification_report(Y_test, prediction))
    #joblib.dump(b_model, 'NB_model.pkl')
    #NB_spam_model = open('NB_model.pkl','rb')
    #clf = joblib.load(NB_spam_model)                   
                      

def random_forest():
    from sklearn.ensemble import RandomForestClassifier
    feature = data.Feed
    target = data.Sentiment
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

    pipe = make_pipeline(TfidfVectorizer(),
                        RandomForestClassifier())
    
    param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
                 'randomforestclassifier__max_features':['sqrt', 'log2']}
    
    rf_model = GridSearchCV(pipe, param_grid, cv=5)
    rf_model.fit(X_train,Y_train)
    
    prediction = rf_model.predict(X_test)
    print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
    print(classification_report(Y_test, prediction))

    #pkl
    joblib.dump(rf_model, 'random_forest.pkl')
    NB_spam_model = open('random_forest.pkl','rb')
    clf = joblib.load(NB_spam_model)
    
    

def super_vector_mach():
    from sklearn.svm import SVC
    feature = data.Feed
    target = data.Sentiment
    X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)
    pipe = make_pipeline(TfidfVectorizer(),SVC())
    param_grid = {'svc__kernel': ['rbf', 'linear', 'poly'],
                 'svc__gamma': [0.1, 1, 10, 100],
                 'svc__C': [0.1, 1, 10, 100]}

    svc_model = GridSearchCV(pipe, param_grid, cv=3)
    svc_model.fit(X_train, Y_train)

    prediction = svc_model.predict(X_test)
    #pkl
    joblib.dump(svc_model, 'super_vector_mach.pkl')
    NB_spam_model = open('super_vector_mach.pkl','rb')
    clf = joblib.load(NB_spam_model)
    print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
    print(classification_report(Y_test, prediction))
#l'utilisation de l'interface    
def Export_fichier_interface(repertoireinit):
    root=tkinter.Tk()
    filename=tkinter.filedialog.askopenfilename(initialdir=repertoireinit,title="choisir votre fichier",filetypes=(("Tous type de fichier","*.*"),))
    root.destroy()
    return filename  
    
pass 
repertoireinit='c:\\'
nom_fich=Export_fichier_interface(repertoireinit)
data = pd.read_excel(nom_fich)
#definir la ponctuation:

data['Feed'] = data['Feed'].apply(preprocess) # appliquerau contenu tashkeel...
#vous pouvez appelez chaque fonctions vous voulez par exemple:
print('1 : pour classification logistic regression')
print('2 : pour classification Bayes')
print('3 : pour classification Support Vector Machines ')
print('4 : pour classification Random Forest')
a=int(input(print('tapez une valeur :')))

if a==1:
    logistic_regression()
if a==2:
    classifier_bayes()
if a==3: 
    super_vector_mach()
if a==4:
    random_forest()
    